In [3]:
import tensorflow as tf

In [4]:
# define the image size and batch size
img_size = (40, 40)
batch_size = 32

# Path to your dataset folder
data_dir = "C:/Users/justi/OneDrive/Desktop/Justin/School/College/S6/S6_Mini_Project/dataset/Mixed_Marks_Dataset"

In [5]:
# use image_dataset_from_directory to load the data and split it into training and testing sets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=img_size,
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=img_size,
    batch_size=batch_size)

Found 18135 files belonging to 8 classes.
Using 14508 files for training.
Found 18135 files belonging to 8 classes.
Using 3627 files for validation.


In [6]:
# inspect the class names and number of classes
class_names = train_ds.class_names
num_classes = len(class_names)
print('Class names:', class_names)
print('Number of classes:', num_classes)

Class names: ['0', '1', '2', '3', '4', '5', '6', '7']
Number of classes: 8


In [7]:
# convert images to grayscale
def grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    return image, label

train_ds = train_ds.map(grayscale)
val_ds = val_ds.map(grayscale)

# normalize pixel values to be between 0 and 1
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [8]:
# create the neural network
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(40, 40, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# The neural network architecture consists of sequential layers, where the output of one layer is passed as input to the next layer.

# The first layer is a convolutional layer with 16 filters of size 3x3, followed by a ReLU activation function. 
# The input shape of the layer is (40, 40, 1), which means that the layer expects input images of size 40x40 with one color channel (grayscale).

# The second layer is a max pooling layer that reduces the spatial dimensions of the output from the previous layer by taking the maximum value in each 2x2 window.

# The third layer is another convolutional layer with 32 filters of size 3x3, followed by a ReLU activation function.

# The fourth layer is another max pooling layer.

# The fifth layer is a flatten layer that flattens the output from the previous layer into a 1D vector.

# The sixth layer is a dense layer with 64 units and a ReLU activation function.

# The seventh and final layer is a dense layer with num_classes units and a softmax activation function, which produces the predicted class probabilities for the input image.

In [21]:
# For visualizing the model

from keras.utils.vis_utils import plot_model
plot_model(model, 
           to_file='model.png',
           show_shapes=True,
           show_dtype=False,
           show_layer_names=True,
           rankdir='TB',
           expand_nested=False,
           dpi=300,
           layer_range=None,
           show_layer_activations=True
          )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [8]:
# from ann_visualizer.visualize import ann_viz;

# ann_viz(model, title="My first neural network")

In [11]:
# compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
# train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


C:\Users\justi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


454/454 [==============================] - 17s 35ms/step - loss: 1.1268 - accuracy: 0.6245 - val_loss: 0.5714 - val_accuracy: 0.8307
Epoch 2/10
454/454 [==============================] - 14s 30ms/step - loss: 0.3685 - accuracy: 0.8890 - val_loss: 0.2722 - val_accuracy: 0.9209
Epoch 3/10
454/454 [==============================] - 15s 32ms/step - loss: 0.2175 - accuracy: 0.9359 - val_loss: 0.2106 - val_accuracy: 0.9391
Epoch 4/10
454/454 [==============================] - 15s 33ms/step - loss: 0.1558 - accuracy: 0.9540 - val_loss: 0.1636 - val_accuracy: 0.9526
Epoch 5/10
454/454 [==============================] - 14s 29ms/step - loss: 0.1119 - accuracy: 0.9703 - val_loss: 0.1394 - val_accuracy: 0.9625
Epoch 6/10
454/454 [==============================] - 16s 35ms/step - loss: 0.0850 - accuracy: 0.9753 - val_loss: 0.1403 - val_accuracy: 0.9584
Epoch 7/10
454/454 [==============================] - 14s 31ms/step - loss: 0.0625 - accuracy: 0.9826 - val_loss: 0.1385 - val_accuracy: 0.9595
Epo

In [13]:
loss, accuracy = model.evaluate(val_ds, verbose=0)

print(f'Testing accuracy: {(accuracy*100):.2f}%')

Validation accuracy: 96.33%


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 38, 38, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 19, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                1

In [15]:
# model.save('MP.h5')

In [16]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(val_ds)

print(cm)

TypeError: confusion_matrix() missing 1 required positional argument: 'y_pred'